In [1]:
import pandas as pd
import numpy as np
import sys, os, re
import matplotlib.pyplot as plt

# from importlib import reload

if re.match(r'^(\w\:\\)|(/)', os.getcwd()):
    # sys.path.insert(0, "c:/Users/dmitr/devs/quantl/")
    # sys.path.insert(0, "c:/Users/dmitr/devs/Qube/")
    sys.path.insert(0, "c:/Users/dmitr/devs/Qube2exp/src/")
    # sys.path.insert(0, "c:/Users/dmitr/devs/Qube2exp/tests/")
    sys.path.insert(0, "c:/Users/dmitr/devs/Qube2exp/")
    print('installed')


# just some handy helpers from old Qube
# from ta_helpers import kama, kama_indicator, apply_to_frame
import tests.qube.ta.utils_for_testing as test

%load_ext cython

T = pd.date_range('2024-01-01 00:00', freq='1Min', periods=100_000)
ds = 1 + (2*np.random.randn(len(T))).cumsum()
data = list(zip(T, ds))

installed
 > Reloaded c:\Users/dmitr/devs/Qube2exp/src\qube\core\series.pyx


## Cython v2 tests (cseries1)

In [3]:
%load_ext autoreload
%autoreload 2

from qube.utils import reload_pyx_module

reload_pyx_module('../../src/qube/core/')
from qube.core.series import TimeSeries
del sys.modules['qube.core.series']
from qube.core.series import (TimeSeries, sma, ema, tema, dema, kama, recognize_time, OHLCV, RollingSum, lag, compare)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
 > Reloaded ../../src/qube/core\series.pyx


## Series example

In [5]:
# %%timeit
ts = TimeSeries('test', '10Min')
ts.update(recognize_time('2024-01-01 00:00'), 1)
ts.update(recognize_time('2024-01-01 00:01'), 5)
ts.update(recognize_time('2024-01-01 00:06'), 2)
ts.update(recognize_time('2024-01-01 00:12'), 3)
ts.update(recognize_time('2024-01-01 00:21'), 4)
ts.update(recognize_time('2024-01-01 00:22'), 5)
ts.update(recognize_time('2024-01-01 00:31'), 6)
ts.update(recognize_time('2024-01-01 00:33'), 7)
ts.update(recognize_time('2024-01-01 00:45'), -12)
ts.update(recognize_time('2024-01-01 00:55'), 12)
ts.update(recognize_time('2024-01-01 01:00'), 12)

1

In [13]:
ts.to_series()

2024-01-01 00:00:00     2.0
2024-01-01 00:10:00     3.0
2024-01-01 00:20:00     5.0
2024-01-01 00:30:00     7.0
2024-01-01 00:40:00   -12.0
2024-01-01 00:50:00    12.0
2024-01-01 01:00:00    12.0
Name: test, dtype: float64

## <font color='orange'>WIP</font>

### Indicators tests

In [151]:
# %%timeit 
ts = TimeSeries('close', '1h')
s1 = sma(ts, 50)
e1 = ema(ts, 50)
ss1 = sma(s1, 50)
ee1 = ema(e1, 50)
t1 = tema(ts, 50)
k1 = kama(ts, 50)
d1 = dema(ts, 50)
test.push(ts, data)

In [152]:
k1s = k1.to_series().dropna()
ik1 = test.apply_to_frame(test.kama, ts.to_series(), 50).dropna()
sum(ik1 - k1.to_series().dropna()) < 1e-8

True

In [113]:
k1 = kama(ts, 50)
k1s = k1.to_series().dropna()
sum(ik1 - k1.to_series().dropna()) < 1e-8

True

In [ ]:
plt.plot(ts.to_series(), lw=0.5)
plt.plot(tema(ts, 50).to_series())
plt.plot(dema(ts, 50).to_series())
plt.plot(t1.to_series())

In [115]:
# %%timeit
x0 = ts.to_series()
pe1 = x0.ewm(span=50, min_periods=50).mean()
pee1 = pe1.ewm(span=50, min_periods=50).mean()
ps1 = x0.rolling(50).mean()
pss1 = ps1.rolling(50).mean()

In [ ]:
plt.plot(x0, lw=0.5)
plt.plot(s1.to_series()); plt.plot(ps1)
plt.plot(ss1.to_series()); plt.plot(pss1)
plt.plot(e1.to_series()); plt.plot(pe1)
plt.plot(ee1.to_series()); plt.plot(pee1)
# plt.plot(s1.to_series()-ps1)

In [116]:
# %%timeit
s1 = sma(ts, 50)
e1 = ema(ts, 50)
ss1 = sma(s1, 50)
ee1 = ema(e1, 50)
k1 = kama(e1, 50)

In [117]:
ts.get_indicators()

{'sma(50)': <qube.core.series.Sma at 0x23b70af9430>,
 'ema(50,init_mean=True)': <qube.core.series.Ema at 0x23b6c65e790>,
 'tema(50,init_mean=True)': <qube.core.series.Tema at 0x23b6be71ca0>,
 'kama(50,2,30)': <qube.core.series.Kama at 0x23b6c4581f0>,
 'dema(50,init_mean=True)': <qube.core.series.Dema at 0x23b6c92a4c0>}

In [ ]:
plt.plot(k1.to_series())

### Indicators wip ... 

In [271]:
ts0 = TimeSeries('close', '1Min')
# em0 = sma(ts0, 3)
# em1 = sma(em0, 3)
k0 = kama(ts0, 6)  
l1 = lag(ts0, 1)
l2 = lag(l1, 1)

for t, v in [
    ('2024-01-01 00:00', 9), ('2024-01-01 00:00', 1),
    ('2024-01-01 00:01', 2), ('2024-01-01 00:01', 3),   ('2024-01-01 00:01', 2),
    ('2024-01-01 00:02', 3),
    ('2024-01-01 00:03', 4),
    ('2024-01-01 00:04', 5), ('2024-01-01 00:04', 5.1), ('2024-01-01 00:04:20', 5),
    ('2024-01-01 00:05', 6), ('2024-01-01 00:05', 7),   ('2024-01-01 00:05', 6),
    ('2024-01-01 00:07', 8), ('2024-01-01 00:07', -1),  ('2024-01-01 00:07', 8),
    ('2024-01-01 00:08', 8),
    ('2024-01-01 00:09', 8),
    ('2024-01-01 00:10', 12),
    ('2024-01-01 00:11', 13),
    ('2024-01-01 00:12', 14),
    ('2024-01-01 00:13', 15),
    ('2024-01-01 00:14', 17),
    ('2024-01-01 00:15', 4),
]:
    ts0.update(recognize_time(t), v)
    # print(l3[0])

In [274]:
test.scols(ts0, l1, l2)

,close,lag(1),lag(1)
2024-01-01 00:00:00,1.0,NaN,NaN
2024-01-01 00:01:00,2.0,1.0,NaN
2024-01-01 00:02:00,3.0,2.0,1.0
2024-01-01 00:03:00,4.0,3.0,2.0
2024-01-01 00:04:00,5.0,4.0,3.0
2024-01-01 00:05:00,6.0,5.0,4.0
2024-01-01 00:07:00,8.0,6.0,5.0
2024-01-01 00:08:00,8.0,8.0,6.0
2024-01-01 00:09:00,8.0,8.0,8.0
2024-01-01 00:10:00,12.0,8.0,8.0


In [276]:
lag(ts0, 2).to_series() == l2.to_series()

2024-01-01 00:00:00    False
2024-01-01 00:01:00    False
2024-01-01 00:02:00     True
2024-01-01 00:03:00     True
2024-01-01 00:04:00     True
2024-01-01 00:05:00     True
2024-01-01 00:07:00     True
2024-01-01 00:08:00     True
2024-01-01 00:09:00     True
2024-01-01 00:10:00     True
2024-01-01 00:11:00     True
2024-01-01 00:12:00     True
2024-01-01 00:13:00     True
2024-01-01 00:14:00     True
2024-01-01 00:15:00     True
dtype: bool

In [ ]:
ts0.get_indicators()

In [252]:
ll2 = lag(lag(ts0, 1),1)

### Caching indicators

In [307]:
ts = TimeSeries('close', '1h')
test.push(ts, data)

In [308]:
sma(ts, 10).to_series()

2024-01-01 00:00:00           NaN
2024-01-01 01:00:00           NaN
2024-01-01 02:00:00           NaN
2024-01-01 03:00:00           NaN
2024-01-01 04:00:00           NaN
                          ...    
2024-03-10 06:00:00    635.948320
2024-03-10 07:00:00    634.558595
2024-03-10 08:00:00    634.599720
2024-03-10 09:00:00    634.414935
2024-03-10 10:00:00    634.240072
Name: sma(10,), Length: 1667, dtype: float64

In [309]:
ema(ts, 10).to_series()
ema(ts, 50).to_series()
tema(ts, 50).to_series()
dema(ts, 50).to_series()

2024-01-01 00:00:00           NaN
2024-01-01 01:00:00           NaN
2024-01-01 02:00:00           NaN
2024-01-01 03:00:00           NaN
2024-01-01 04:00:00           NaN
                          ...    
2024-03-10 06:00:00    630.626537
2024-03-10 07:00:00    633.594281
2024-03-10 08:00:00    636.005011
2024-03-10 09:00:00    638.245717
2024-03-10 10:00:00    640.161844
Name: dema(50,init_mean=True), Length: 1667, dtype: float64

In [310]:
ts.get_indicators()

{'sma(10,)': <qube.core.series.Sma at 0x1e30d2dbf20>,
 'ema(10,init_mean=True)': <qube.core.series.Ema at 0x1e30ca170d0>,
 'ema(50,init_mean=True)': <qube.core.series.Ema at 0x1e30d29f700>,
 'tema(50,init_mean=True)': <qube.core.series.Tema at 0x1e30d29f790>,
 'dema(50,init_mean=True)': <qube.core.series.Dema at 0x1e30cb502c0>}

### OHLC tests

In [367]:
ohlc = OHLCV('1Min')
s1 = sma(ohlc.close, 10)
for t, v in [
    ('2024-01-01 00:00', 9), ('2024-01-01 00:00', 1),
    ('2024-01-01 00:01', 2), ('2024-01-01 00:01', 3),   ('2024-01-01 00:01', 2),
    ('2024-01-01 00:02', 3),
    ('2024-01-01 00:03', 4),
    ('2024-01-01 00:04', 5), ('2024-01-01 00:04', 5.1), ('2024-01-01 00:04:20', 5),
    ('2024-01-01 00:05', 6), ('2024-01-01 00:05', 7),   ('2024-01-01 00:05', 6),
    ('2024-01-01 00:07', 8), ('2024-01-01 00:07', -1),  ('2024-01-01 00:07', 8),
    ('2024-01-01 00:08', 8),
    ('2024-01-01 00:09', 8),
    ('2024-01-01 00:10', 12), ('2024-01-01 00:10:01', 21),('2024-01-01 00:10:30', 1),('2024-01-01 00:10:31', 5),
    ('2024-01-01 00:11', 13),
    ('2024-01-01 00:12', 14),
    ('2024-01-01 00:13', 15),
    ('2024-01-01 00:14', 17),
    ('2024-01-01 00:15', 4), ('2024-01-01 00:15:10', 12),
]:
    ohlc.update(recognize_time(t), v, 1)

In [369]:
r1 = RollingSum(3)

In [381]:
r1.update(1, 1); print(r1)
# r1.update(1, 1); print(r1)
# r1.update(1, 0); print(r1)

rs[3] = [1. 1. 1.] @ 0 -> 0


In [ ]:
ohlc.to_series()

In [385]:
print(str(ohlc), str(s1))

OHLCV[1Min] | 15 records
  2024-01-01T00:15:00.000000000 {o:4.000000 | h:12.000000 | l:4.000000 | c:12.000000 | v:2.000000}
  2024-01-01T00:14:00.000000000 {o:17.000000 | h:17.000000 | l:17.000000 | c:17.000000 | v:1.000000}
  2024-01-01T00:13:00.000000000 {o:15.000000 | h:15.000000 | l:15.000000 | c:15.000000 | v:1.000000}
   .......... 
  2024-01-01T00:02:00.000000000 {o:3.000000 | h:3.000000 | l:3.000000 | c:3.000000 | v:1.000000}
  2024-01-01T00:01:00.000000000 {o:2.000000 | h:3.000000 | l:2.000000 | c:2.000000 | v:3.000000}
  2024-01-01T00:00:00.000000000 {o:9.000000 | h:9.000000 | l:1.000000 | c:1.000000 | v:2.000000}
 sma(10)[1Min] | 15 records
  2024-01-01T00:15:00.000000000 10.6
  2024-01-01T00:14:00.000000000 9.9
  2024-01-01T00:13:00.000000000 8.6
   .......... 
  2024-01-01T00:02:00.000000000 nan
  2024-01-01T00:01:00.000000000 nan
  2024-01-01T00:00:00.000000000 nan



In [386]:
s1.to_series()

2024-01-01 00:00:00     NaN
2024-01-01 00:01:00     NaN
2024-01-01 00:02:00     NaN
2024-01-01 00:03:00     NaN
2024-01-01 00:04:00     NaN
2024-01-01 00:05:00     NaN
2024-01-01 00:07:00     NaN
2024-01-01 00:08:00     NaN
2024-01-01 00:09:00     NaN
2024-01-01 00:10:00     NaN
2024-01-01 00:11:00     6.2
2024-01-01 00:12:00     7.4
2024-01-01 00:13:00     8.6
2024-01-01 00:14:00     9.9
2024-01-01 00:15:00    10.6
Name: sma(10), dtype: float64

In [388]:
sma(ohlc.close, 10).to_series()

2024-01-01 00:00:00     NaN
2024-01-01 00:01:00     NaN
2024-01-01 00:02:00     NaN
2024-01-01 00:03:00     NaN
2024-01-01 00:04:00     NaN
2024-01-01 00:05:00     NaN
2024-01-01 00:07:00     NaN
2024-01-01 00:08:00     NaN
2024-01-01 00:09:00     NaN
2024-01-01 00:10:00     5.0
2024-01-01 00:11:00     6.2
2024-01-01 00:12:00     7.4
2024-01-01 00:13:00     8.6
2024-01-01 00:14:00     9.9
2024-01-01 00:15:00    10.6
Name: sma(10), dtype: float64

In [389]:
sma(ohlc.close, 8).to_series()

2024-01-01 00:00:00       NaN
2024-01-01 00:01:00       NaN
2024-01-01 00:02:00       NaN
2024-01-01 00:03:00       NaN
2024-01-01 00:04:00       NaN
2024-01-01 00:05:00       NaN
2024-01-01 00:07:00       NaN
2024-01-01 00:08:00     4.625
2024-01-01 00:09:00     5.500
2024-01-01 00:10:00     5.875
2024-01-01 00:11:00     7.125
2024-01-01 00:12:00     8.375
2024-01-01 00:13:00     9.625
2024-01-01 00:14:00    11.000
2024-01-01 00:15:00    11.500
Name: sma(8), dtype: float64

## How to work with indicators in strategy

In [16]:
xs = TimeSeries('close', '1h')
compare(lag(xs, 1), xs)
test.push(xs, data)

C:\Users\dmitr\AppData\Local\Temp\ipykernel_16112\2738742798.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  compare(lag(xs, 1), xs)
C:\Users\dmitr\AppData\Local\Temp\ipykernel_16112\2738742798.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  compare(lag(xs, 1), xs)


In [17]:
test.scols(xs.to_series(), lag(xs, 1).to_series(), compare(xs, lag(xs, 1)).to_series())

,close,lag(1),compare(lag(1))
2024-01-01 00:00:00,-14.900003,NaN,NaN
2024-01-01 01:00:00,-3.885655,-14.900003,1.0
2024-01-01 02:00:00,1.508510,-3.885655,1.0
2024-01-01 03:00:00,14.622315,1.508510,1.0
2024-01-01 04:00:00,-2.391985,14.622315,-1.0
...,...,...,...
2024-03-10 06:00:00,-331.436662,-354.194934,1.0
2024-03-10 07:00:00,-333.803098,-331.436662,-1.0
2024-03-10 08:00:00,-339.362240,-333.803098,-1.0
2024-03-10 09:00:00,-348.400754,-339.362240,-1.0


In [18]:
xs.update(pd.Timestamp('2024-03-10 15:00:01').asm8, 1)
test.scols(xs.to_series(), lag(xs, 1).to_series(), compare(xs, lag(xs, 1)).to_series())

,close,lag(1),compare(lag(1))
2024-01-01 00:00:00,-14.900003,NaN,NaN
2024-01-01 01:00:00,-3.885655,-14.900003,1.0
2024-01-01 02:00:00,1.508510,-3.885655,1.0
2024-01-01 03:00:00,14.622315,1.508510,1.0
2024-01-01 04:00:00,-2.391985,14.622315,-1.0
...,...,...,...
2024-03-10 07:00:00,-333.803098,-331.436662,-1.0
2024-03-10 08:00:00,-339.362240,-333.803098,-1.0
2024-03-10 09:00:00,-348.400754,-339.362240,-1.0
2024-03-10 10:00:00,-339.151931,-348.400754,1.0


In [402]:
lag(xs, 1).name

'lag(1)'

In [410]:
for i in range(len(xs) - 1):
    if (ema(xs, 15)[i] > ema(xs, 50)[i]) and (ema(xs, 15)[i+1] < ema(xs, 50)[i+1]):
        print(pd.Timestamp(xs.times[i]))

2024-03-08 12:00:00
2024-03-06 05:00:00
2024-03-01 23:00:00
2024-02-26 16:00:00
2024-02-24 17:00:00
2024-02-21 10:00:00
2024-02-18 03:00:00
2024-02-17 03:00:00
2024-02-14 04:00:00
2024-02-10 04:00:00
2024-02-02 09:00:00
2024-02-01 03:00:00
2024-01-27 01:00:00
2024-01-22 18:00:00
2024-01-19 04:00:00
2024-01-10 20:00:00
2024-01-07 03:00:00
2024-01-04 11:00:00


In [282]:
xs.get_indicators()

{'kama(51,2,30)': <qube.core.series.Kama at 0x23b679c8700>,
 'ema(15,init_mean=True)': <qube.core.series.Ema at 0x23b70dd7b80>,
 'ema(50,init_mean=True)': <qube.core.series.Ema at 0x23b70dd7ee0>}

In [314]:
s0 = xs.to_series()
s1 = ema(xs, 15).to_series()
r = pd.concat((s0, s1), axis=1)

def cmp(a,b) -> int:
    if np.isnan(a) or np.isnan(b):
        return np.nan
    return +1 if a > b else -1 if a < b else 0
for t,(a,b) in zip(r.index, r.values):
    print(t, cmp(a,b))

In [ ]:
class IStrategy:

    def trade(self, position: float, group=None, stop=None, take=None, limit=None, cancel=False):
        pass

    def cancel(self, group=None):
        pass

    def position(self) -> float:
        """
        Returns open position
        """
        return 0.0


class CrossTest(IStrategy):

    def __init__(self) -> None:
        pass

    def on_data(self, xs: OHLCV, is_bar_closed: bool):
        if is_bar_closed:
            fast = ema(xs, 15)
            slow = ema(xs, 50)
            b = xs[0]

            if (fast[0] > slow[0]) and (fast[1] < slow[1]):
                self.trade(+1, 'long', stop=b.low, cancel=True)

            elif (fast[0] < slow[0]) and (fast[1] > slow[1]):
                self.trade(-1, 'short', stop=b.high, cancel=True)
            